In [ ]:
# Imports
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, TwoLocal, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit.quantum_info import SparsePauliOp
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
import time
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel
from typing import List, Dict, Optional, Tuple
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.passes import BasicSwap, TrivialLayout, SabreSwap, SabreLayout, UnrollCustomDefinitions, BasisTranslator
from qiskit import transpile

In [ ]:
# Quantum Kernel Circuits with different entanglement strategies for ZZFeatureMap

def generate_zz_linear_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

def generate_zz_circular_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='circular', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

def generate_zz_sca_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='sca', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

def generate_zz_pairwise_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='pairwise', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

# QNN circuits with fixed ZZFeatureMap (linear entanglement) but varying TwoLocal entanglement

def generate_qnn_linear(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with linear entanglement
    ansatz = TwoLocal(num_qubits, entanglement='linear', reps=ansatz_reps)
    
    
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

def generate_qnn_circular(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with circular entanglement
    ansatz = TwoLocal(num_qubits, entanglement='circular', reps=ansatz_reps)
    
   
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

def generate_qnn_sca(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with sca entanglement
    ansatz = TwoLocal(num_qubits, entanglement='sca', reps=ansatz_reps)
    
    
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

def generate_qnn_pairwise(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with pairwise entanglement
    ansatz = TwoLocal(num_qubits, entanglement='pairwise', reps=ansatz_reps)
    
    
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

In [ ]:
def create_tree_tensor_network(num_qubits):
    """
    Create a Tree Tensor Network (TTN) quantum circuit with logarithmic depth.
    Each layer has exactly half the number of gates as the previous layer.
    """
    # Must have a power of 2 number of qubits for a balanced tree
    if num_qubits & (num_qubits - 1) != 0:
        # If not a power of 2, round up to the next power of 2
        next_power = 2 ** np.ceil(np.log2(num_qubits))
        num_qubits = int(next_power)
    

    qc = QuantumCircuit(num_qubits)
    
    # Initialize all qubits with Hadamard gates
    for i in range(num_qubits):
        qc.h(i)
    
    qc.barrier()
    
    # Calculate number of layers (logarithmic depth)
    num_layers = int(np.log2(num_qubits))
    
    # Working qubits at each level
    active_qubits = list(range(num_qubits))
    
    # Create each layer of the tree
    for layer in range(num_layers):
        num_pairs = len(active_qubits) // 2
        new_active_qubits = []
        
        # Apply entangling gates between pairs and keep track of parent qubits
        for i in range(num_pairs):
            q1 = active_qubits[2*i]
            q2 = active_qubits[2*i + 1]
            
            # Create entanglement between pair
            qc.cx(q1, q2)
            
            qc.ry(np.pi/4, q1)
            qc.rz(np.pi/6, q2)
            
            # The first qubit of each pair becomes the "parent" for the next layer
            new_active_qubits.append(q1)
        
        qc.barrier()
        active_qubits = new_active_qubits
    
    return qc

In [ ]:
# GHZ State Circuit
def generate_ghz_state(num_qubits):

    circuit = QuantumCircuit(num_qubits)
    
    # Apply Hadamard to the first qubit
    circuit.h(0)
    
    # Apply CNOTs to entangle all qubits
    for i in range(num_qubits - 1):
        circuit.cx(i, i + 1)
    
    return circuit

In [ ]:
class CustomTopologyBackend(BackendV2):
    """Custom backend with configurable topology, error rates, and gate times."""
    
    def __init__(self, num_qubits, topology_type='grid', 
                 single_gate_error=7.42e-5, two_gate_error=7e-4, readout_error=1.67e-4,
                 t1_time=1.2e-3, t2_time=1.16e-3,
                 single_gate_time=7.9e-9, two_gate_time=30.0e-9):
        # Generate a name for the backend
        backend_name = f"{topology_type}_{num_qubits}q_custom"
        super().__init__(name=backend_name)
        
        # Configuration parameters
        self._num_qubits = num_qubits
        self.topology_type = topology_type
        self.single_gate_error = single_gate_error
        self.two_gate_error = two_gate_error
        self.readout_error = readout_error
        self.t1_time = t1_time
        self.t2_time = t2_time
        self.single_gate_time = single_gate_time
        self.two_gate_time = two_gate_time
        
        # Coupling map for the specified topology
        self._coupling_map = self._create_topology(topology_type, num_qubits)
    
    def _create_topology(self, topology_type, num_qubits):
        """Create a coupling map based on topology type."""
        edges = []
        
        if topology_type == 'linear':
            edges = [(i, i+1) for i in range(num_qubits-1)]
            
        elif topology_type == 'ring':
            edges = [(i, i+1) for i in range(num_qubits-1)]
            edges.append((num_qubits-1, 0))
            
        elif topology_type == 'grid':
            size = int(np.ceil(np.sqrt(num_qubits)))
            for i in range(num_qubits):
                row, col = i // size, i % size
                # Horizontal connections
                if col < size-1 and i+1 < num_qubits:
                    edges.append((i, i+1))
                # Vertical connections
                if row < size-1 and i+size < num_qubits:
                    edges.append((i, i+size))
                    
        elif topology_type == 'star':
            edges = [(0, i) for i in range(1, num_qubits)]
            
        else:
            raise ValueError(f"Unknown topology type: {topology_type}")
        
        # Create bidirectional edges for bidirection implementation of 2 qubit gates
        full_edges = edges + [(j, i) for i, j in edges]
        return CouplingMap(full_edges)
    
    def properties(self):
        """
        Return a dictionary of properties.
        """
        props = {
            'backend_name': self.name,
            'backend_version': '1.0.0',
            'qubits': [],
            'gates': [],
        }
        
        # Add qubit properties
        for i in range(self._num_qubits):
            props['qubits'].append([
                {'name': 'T1', 'value': self.t1_time * 1e6, 'unit': 'µs'},
                {'name': 'T2', 'value': self.t2_time * 1e6, 'unit': 'µs'},
                {'name': 'readout_error', 'value': self.readout_error, 'unit': ''}
            ])
        
        # Add gate properties
        for i in range(self._num_qubits):
            for gate_name in ['u3']:
                props['gates'].append({
                    'gate': gate_name,
                    'parameters': [
                        {'name': 'gate_error', 'value': self.single_gate_error, 'unit': ''},
                        {'name': 'gate_length', 'value': self.single_gate_time * 1e9, 'unit': 'ns'}
                    ],
                    'qubits': [i]
                })
        
        # Add two-qubit gate properties
        for [q1, q2] in self._coupling_map.get_edges():
            props['gates'].append({
                'gate': 'cx',
                'parameters': [
                    {'name': 'gate_error', 'value': self.two_gate_error, 'unit': ''},
                    {'name': 'gate_length', 'value': self.two_gate_time * 1e9, 'unit': 'ns'}
                ],
                'qubits': [q1, q2]
            })
        
        return props
    
    # Required abstract methods from BackendV2
    
    def _default_options(self):
        """Return default options for the backend."""
        return Options(shots=1024, memory=False)
    
    @property
    def target(self):
        """Return the target for compilation."""
        return None  # Simplified for this example
        
    @property
    def max_circuits(self):
        """Maximum number of circuits."""
        return None
        
    @property
    def coupling_map(self):
        """Return the coupling map."""
        return self._coupling_map
        
    @property
    def operation_names(self):
        """Return the operation names."""
        return ['u3', 'cx']
        
    @property
    def num_qubits(self):
        """Return the number of qubits."""
        return self._num_qubits


def backend_to_errors(backend):
    """
    Extract error parameters from backend properties for fidelity calculation.
    """
    properties = backend.properties()
    
    error_params = {}
    gate_times = {}
    t1_times = {}
    t2_times = {}
    
    # Extract T1 and T2 times
    for i, qubit_props in enumerate(properties['qubits']):
        t1_prop = next((prop for prop in qubit_props if prop['name'] == 'T1'), None)
        t2_prop = next((prop for prop in qubit_props if prop['name'] == 'T2'), None)
        
        if t1_prop and 'value' in t1_prop:
            t1_times[i] = t1_prop['value'] * 1e-6  # Convert from µs to s
        if t2_prop and 'value' in t2_prop:
            t2_times[i] = t2_prop['value'] * 1e-6  # Convert from µs to s
    
    # Extract gate errors and times
    for gate_prop in properties['gates']:
        gate_name = gate_prop['gate']
        qubits = gate_prop['qubits']
        
        # Find error and time parameters
        error_param = next((param['value'] for param in gate_prop['parameters'] 
                           if param['name'] == 'gate_error'), 0)
        gate_time = next((param['value'] * 1e-9 for param in gate_prop['parameters']  # Convert from ns to s
                         if param['name'] == 'gate_length'), 0)

        if len(qubits) == 1:
            key = f"{gate_name}{qubits[0]}"
        else:
            key = f"{gate_name}{qubits[0]}_{qubits[1]}"
        
        error_params[key] = error_param
        gate_times[key] = gate_time
    
    return error_params, gate_times, t1_times, t2_times

In [ ]:
def depolarizing_fidelity(circ, backend, p=0.5, include_t1t2=False):
    """
    Estimate fidelity using the depolarizing noise model from the paper arXiv:2503.06693
    """
    # Extract error parameters from the backend
    error_params, gate_times, t1_times, t2_times = backend_to_errors(backend)
    
    # Initialize qubit-wise fidelity tracking for all qubits in the backend
    qubits_fidelity = [1.0 for _ in range(backend.num_qubits)]
    
    # Convert circuit to layer-based representation
    dag = circuit_to_dag(circ)
    layers = [dag_to_circuit(layer['graph']) for layer in dag.layers()]
    
    # Track which qubits are measured (for final fidelity calculation)
    measured_qubits = set()
    
    # Track execution time for T1/T2 calculations
    exec_time = [0]
    
    # Process each layer
    for layer in layers:
        slowest_gate = 0
        
        # Process gates in this layer
        for gate in layer:

            gate_name = f'{gate.name}{circ.find_bit(gate.qubits[0]).index}'
            if gate.operation.num_qubits == 2:
                gate_name += f'_{circ.find_bit(gate.qubits[1]).index}'
            
            # Track measured qubits for final fidelity calculation
            if 'measure' in gate_name:
                measured_qubits.add(circ.find_bit(gate.qubits[0]).index)
                continue
            
            # Track execution time for this layer
            if gate_name in gate_times:
                slowest_gate = max(slowest_gate, gate_times[gate_name])
            
            # Single-qubit gates fidelity
            if gate.operation.num_qubits == 1:
                gate_key = f'{gate.operation.name}{circ.find_bit(gate.qubits[0]).index}'
                if gate_key in error_params:
                    qubit = circ.find_bit(gate.qubits[0]).index
                    l = error_params[gate_key]
                    qubits_fidelity[qubit] = (1-l) * qubits_fidelity[qubit] + (1-p) * l/2
            
            # Two-qubit gates fidelity
            else:
                gate_key = f'{gate.operation.name}{circ.find_bit(gate.qubits[0]).index}_{circ.find_bit(gate.qubits[1]).index}'
                if gate_key in error_params:
                    qubit_1 = circ.find_bit(gate.qubits[0]).index
                    qubit_2 = circ.find_bit(gate.qubits[1]).index
                    
                    fid_1 = qubits_fidelity[qubit_1]
                    fid_2 = qubits_fidelity[qubit_2]
                    
                    l = error_params[gate_key]
                    
                    # Calculate η parameter from the paper
                    c = 1/2 * (np.sqrt((1-l) * (fid_1 + fid_2)**2 + l) - np.sqrt(1-l) * (fid_1 + fid_2))
                    
                    # Update fidelities
                    fid_1 = np.sqrt(1-l) * fid_1 + (1-p) * c
                    fid_2 = np.sqrt(1-l) * fid_2 + (1-p) * c
                    
                    qubits_fidelity[qubit_1] = fid_1
                    qubits_fidelity[qubit_2] = fid_2
        
        # Update execution time
        exec_time.append(exec_time[-1] + slowest_gate)
        
        # Apply T1/T2 coherence effects (user enabled)
        if include_t1t2:
            # Time for this layer
            layer_time = slowest_gate  
            
            for q in range(len(qubits_fidelity)):
                if q in t1_times and q in t2_times:
                    # Apply T1 decay
                    t1_factor = np.exp(-layer_time / t1_times[q])
                    
                    # Apply T2 decay using formula from paper (Equation 41)
                    t2_factor = 0.5 * np.exp(-layer_time / t2_times[q]) + 0.5
                    
                    # Combine decoherence effects
                    qubits_fidelity[q] *= t1_factor * t2_factor
    
    # If we have measured qubits, use them for final fidelity, or use all qubits
    if measured_qubits:
        return np.prod([qubits_fidelity[i] for i in measured_qubits])
    else:
        return np.prod(qubits_fidelity)

In [ ]:
def analyze_circuit_with_fidelity(circuit, backend, p_ent=0.5, include_t1t2=False, basis_gates=None):
    """
    Transpiles a quantum circuit and analyzes its metrics including fidelity.
    """
    from qiskit.transpiler import PassManager
    from qiskit.transpiler.passes import SabreSwap, TrivialLayout, UnrollCustomDefinitions, BasisTranslator
    import time
    
    # Set default basis gates if not provided
    if basis_gates is None:
        basis_gates = ['cx', 'u3']
    
    # Get coupling map from backend
    coupling_map = backend.coupling_map
    
    # Configure transpiler passes
    routing = SabreSwap(coupling_map=coupling_map, trials=1, heuristic='basic')
    placement = TrivialLayout(coupling_map=coupling_map)
    
    # Create passes for basis gate transformation
    unroll = UnrollCustomDefinitions(sel) 
    basis_translator = BasisTranslator(sel, basis_gates)
    
    # Create pass manager with all passes
    pm = PassManager()
    if basis_gates:
        pm.append([placement, routing, unroll, basis_translator])
    else:
        pm.append([placement, routing])
    
    # Measure transpilation time
    init_time = time.monotonic()
    transpiled_circuit = pm.run(circuit)
    exec_time = time.monotonic() - init_time
    
    # Extract error parameters and gate times from backend
    error_params, gate_times, t1_times, t2_times = backend_to_errors(backend)
    
    # Calculate fidelity using the depolarizing model
    fidelity = depolarizing_fidelity(
        transpiled_circuit, 
        backend, 
        p=p_ent, 
        include_t1t2=include_t1t2
    )
    
    # Return all metrics
    return {
        'transpiled_circuit': transpiled_circuit,
        'transpilation_time': exec_time,
        'fidelity': fidelity,
        'error_model': {
            'single_gate_error': backend.single_gate_error if hasattr(backend, 'single_gate_error') else None,
            'two_gate_error': backend.two_gate_error if hasattr(backend, 'two_gate_error') else None,
            'readout_error': backend.readout_error if hasattr(backend, 'readout_error') else None,
            't1_time': backend.t1_time if hasattr(backend, 't1_time') else None,
            't2_time': backend.t2_time if hasattr(backend, 't2_time') else None,
            'single_gate_time': backend.single_gate_time if hasattr(backend, 'single_gate_time') else None,
            'two_gate_time': backend.two_gate_time if hasattr(backend, 'two_gate_time') else None,
            'gate_times': gate_times,  # Added gate times dictionary
            'p_ent': p_ent
        }
    }

In [ ]:
def plot_fidelity_vs_qubits():
    """
    Creates plots of fidelity vs number of qubits for all circuit types
    on different processor topologies.
    """
    
    mpl.rcParams['font.size'] = 9
    mpl.rcParams['axes.labelweight'] = 'bold'
    mpl.rcParams['axes.titleweight'] = 'bold'
    
    # Circuit types to analyze
    circuit_types = {
        'Kernel Linear': lambda n: generate_zz_linear_kernel(n, reps=1),
        'Kernel Circular': lambda n: generate_zz_circular_kernel(n, reps=1),
        'Kernel SCA': lambda n: generate_zz_sca_kernel(n, reps=1),
        'Kernel Pairwise': lambda n: generate_zz_pairwise_kernel(n, reps=1),
        'QNN Linear': lambda n: generate_qnn_linear(n, ansatz_reps=2, feature_map_reps=1),
        'QNN Circular': lambda n: generate_qnn_circular(n, ansatz_reps=2, feature_map_reps=1),
        'QNN SCA': lambda n: generate_qnn_sca(n, ansatz_reps=2, feature_map_reps=1),
        'QNN Pairwise': lambda n: generate_qnn_pairwise(n, ansatz_reps=2, feature_map_reps=1),
        'TTN': lambda n: create_tree_tensor_network(n),
        'GHZ': lambda n: generate_ghz_state(n)
    }
    
    # Backend topologies
    backend_topologies = ['Linear', 'Ring', 'Grid', 'Star']
    
    # Qubit ranges for each circuit type
    ttn_qubits = [4, 8, 16, 32, 64]
    regular_qubits = list(range(10, 101, 10))
    
    markers = {name: ('None' if name == 'GHZ' else style) for name, style in 
              zip(circuit_types.keys(), ['o', 's', 'D', 'v', '^', '<', '>', 'p', '*', 'X'])}
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(circuit_types)))
    
    # Line styles - dotted for GHZ, solid for others
    line_styles = {name: (':' if name == 'GHZ' else '-') for name in circuit_types.keys()}

    line_widths = {name: (2.5 if name == 'GHZ' else 1.5) for name in circuit_types.keys()}

    fig, axes = plt.subplots(1, 4, figsize=(12, 2.8))

    circuit_names = list(circuit_types.keys())
    
    # Store all lines for the legend
    handles = []
    labels = []
    

    for t, topology in enumerate(backend_topologies):
        ax = axes[t]
        
        results = {}

        for circuit_idx, circuit_name in enumerate(circuit_names):
            circuit_generator = circuit_types[circuit_name]
            
            # Qubit range to use
            qubit_range = ttn_qubits if circuit_name == 'TTN' else regular_qubits
            
            fidelities = []
            valid_qubits = []
            
            for num_qubits in qubit_range:
                # Create backend with topology
                backend = CustomTopologyBackend(
                    num_qubits=num_qubits,
                    topology_type=topology.lower()
                )
                
                try:
                    # Generate circuit
                    circuit = circuit_generator(num_qubits)
                    
                    # Decompose circuits with ZZFeatureMap or TwoLocal
                    needs_decompose = any(name in circuit_name for name in ['Kernel', 'QNN'])
                    if needs_decompose:
                        circuit = circuit.decompose()
                    
                    # Calculate fidelity
                    analysis = analyze_circuit_with_fidelity(
                        circuit=circuit,
                        backend=backend,
                        p_ent=0.5,
                        include_t1t2=True # Analyse the effects of decoherence
                    )
                    
                    fidelities.append(analysis['fidelity'])
                    valid_qubits.append(num_qubits)
                    
                except Exception as e:
                    print(f"Error with {circuit_name} for {num_qubits} qubits on {topology} topology: {str(e)}")
            
            # Plotting
            if valid_qubits:
                line = ax.plot(valid_qubits, fidelities, 
                         marker=markers[circuit_name],
                         linestyle=line_styles[circuit_name],
                         linewidth=line_widths[circuit_name],
                         color=colors[circuit_idx],
                         markersize=4,
                         label=circuit_name)
                
                if t == 0:
                    handles.append(line[0])
                    labels.append(circuit_name)

        ax.set_xlabel('Number of Qubits', fontweight='bold', fontsize=9)
        if t == 0:
            ax.set_ylabel('Fidelity', fontweight='bold', fontsize=9)
        ax.set_title(f'{topology} Topology', fontweight='bold', fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.3, linewidth=0.5)
        ax.set_ylim([0, 1.05]) 
        #ax.set_xlim([0, 100])

        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_linewidth(0.5)

        ax.tick_params(axis='both', labelsize=7)

    leg = fig.legend(handles, labels, 
                   loc='lower center', 
                   bbox_to_anchor=(0.5, 0.07),
                   ncol=10,
                   fontsize=8,  # Smaller for the reduced height 
                   frameon=True,
                   framealpha=0.8,
                   columnspacing=0.7, 
                   handletextpad=0.4)

    plt.subplots_adjust(bottom=0.07, wspace=0.25)

    plt.tight_layout(rect=[0, 0.16, 1, 0.97])
    
    # Save the figure in multiple formats
    plt.savefig('Fidelity vs qubits (100).pdf', bbox_inches='tight', dpi=600)
    # plt.savefig('Fidelity vs qubits (100).svg', bbox_inches='tight', dpi=600)
    # plt.savefig('Fidelity vs qubits (100).png', bbox_inches='tight', dpi=600)
    
    plt.show()
    
    return fig, axes

# Execute the function
fig, axes = plot_fidelity_vs_qubits()